In [59]:
import os
from pathlib import Path

# Change to repo root
repo_root = Path(r"d:\ML PROJECTS\End-to-End-Machine-Learning-Project-with-MLflow")
os.chdir(repo_root)
print(f"Working directory: {os.getcwd()}")

Working directory: d:\ML PROJECTS\End-to-End-Machine-Learning-Project-with-MLflow


In [27]:
import os

In [28]:
%pwd

'd:\\ML PROJECTS'

In [29]:
os.chdir("../")

In [53]:
%pwd

'd:\\'

In [31]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [49]:
import importlib
import MLproject.constants
importlib.reload(MLproject.constants)
from MLproject.constants import *
from MLproject.utils.common import read_yaml, create_directories

In [56]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = None,
        params_filepath = None,
        schema_filepath = None):
        
        # Get repo root from current working directory (go up one level from research)
        repo_root = Path.cwd().parent if Path.cwd().name == 'research' else Path.cwd()
        
        if config_filepath is None:
            from MLproject.constants import CONFIG_FILE_PATH
            config_filepath = CONFIG_FILE_PATH
        if params_filepath is None:
            from MLproject.constants import PARAMS_FILE_PATH
            params_filepath = PARAMS_FILE_PATH
        if schema_filepath is None:
            from MLproject.constants import SCHEMA_FILE_PATH
            schema_filepath = SCHEMA_FILE_PATH

        # Convert relative paths to absolute paths relative to repo root
        if not Path(config_filepath).is_absolute():
            config_filepath = repo_root / config_filepath
        if not Path(params_filepath).is_absolute():
            params_filepath = repo_root / params_filepath
        if not Path(schema_filepath).is_absolute():
            schema_filepath = repo_root / schema_filepath

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # Convert config paths to absolute paths
        artifacts_root = self.config.artifacts_root
        if not Path(artifacts_root).is_absolute():
            artifacts_root = repo_root / artifacts_root
        create_directories([artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        repo_root = Path.cwd().parent if Path.cwd().name == 'research' else Path.cwd()
        config = self.config.data_ingestion
        
        # Convert paths to absolute relative to repo root
        root_dir = config.root_dir
        if not Path(root_dir).is_absolute():
            root_dir = repo_root / root_dir
        
        local_data_file = config.local_data_file
        if not Path(local_data_file).is_absolute():
            local_data_file = repo_root / local_data_file
        
        unzip_dir = config.unzip_dir
        if not Path(unzip_dir).is_absolute():
            unzip_dir = repo_root / unzip_dir

        create_directories([root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=root_dir,
            source_URL=config.source_URL,
            local_data_file=local_data_file,
            unzip_dir=unzip_dir
        )

        return data_ingestion_config

In [34]:
import os
import urllib.request as request # request to download data from url
import zipfile
from MLproject import logger
from MLproject.utils.common import get_size

In [35]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [60]:
try:
    config = ConfigurationManager()
    print("✓ ConfigurationManager created")
    data_ingestion_config = config.get_data_ingestion_config()
    print("✓ Data ingestion config retrieved")
    data_ingestion = DataIngestion(config=data_ingestion_config)
    print("✓ DataIngestion instance created")
    data_ingestion.download_file()
    print("✓ File downloaded")
    data_ingestion.extract_zip_file()
    print("✓ File extracted")
except Exception as e:
    print(f"❌ Error: {type(e).__name__}")
    print(f"Message: {str(e)}")
    import traceback
    traceback.print_exc()

[2026-02-01 21:00:16,705: INFO: common: yaml file: D:\ML PROJECTS\End-to-End-Machine-Learning-Project-with-MLflow\config\config.yaml loaded successfully]
[2026-02-01 21:00:16,707: INFO: common: yaml file: D:\ML PROJECTS\End-to-End-Machine-Learning-Project-with-MLflow\params.yaml loaded successfully]
[2026-02-01 21:00:16,711: INFO: common: yaml file: D:\ML PROJECTS\End-to-End-Machine-Learning-Project-with-MLflow\schema.yaml loaded successfully]
[2026-02-01 21:00:16,714: INFO: common: created directory at: d:\ML PROJECTS\End-to-End-Machine-Learning-Project-with-MLflow\artifacts]
✓ ConfigurationManager created
[2026-02-01 21:00:16,717: INFO: common: created directory at: d:\ML PROJECTS\End-to-End-Machine-Learning-Project-with-MLflow\artifacts\data_ingestion]
✓ Data ingestion config retrieved
✓ DataIngestion instance created
[2026-02-01 21:00:17,943: INFO: 2366278714: d:\ML PROJECTS\End-to-End-Machine-Learning-Project-with-MLflow\artifacts\data_ingestion\data.zip download! with following i